# AE425 - Engine Design Project - Mikhail LaMay

In [3]:
# Python Packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

# Create Engine Component Dataframe
engine_comp = [(1.00,1.40,1716), 
               (0.97,1.4,1716), 
               (0.90,1.4,1716), 
               (0.00,1.33,1775), 
               (0.91,1.33,1775), 
               (1.00,1.33,1775), 
               (1.00,1.40,1716)]

# Create DataFrame object from a list of tuples
dfObj = pd.DataFrame(engine_comp, columns = ['Eta_Poly', 'Gamma' , 'R'], 
                     index=['Inlet', 'Fan','Compressor','Combustor','Turbine',
                            'Nozzle (Core)','Nozzle (Bypass)'])

# Calculate C_p for each section of Turbofan in Dataframe
R_ls = dfObj['R']
gamma_ls = dfObj['Gamma'] 
C_p = R_ls*(gamma_ls/(gamma_ls-1))
dfObj['C_p'] = C_p

dfObj

# Thermodynamic Properties of Air:
R_ssl_air = 1716 # ft-lbs/sl-R
C_P_air = 6006 # ft-lbs/sl-R
gamma_air = 1.4

# Standard Sea Level Conditions of Air:
Press_ssl = 2116 # lbs/ft^2
den_ssl = 0.002377 # sl/ft^3
Temp_ssl = 518.67 # R

# Gravity
g = 32.2 # ft/s^2

#----------# Start of Function #-------------------------------------------------------------------------------------------#

def EngineDesign(alt, M0, BPR, FPR, OPR, T_t4, D_fan):

#----------# Atmosphere #--------------------------------------------------------------------------------------------------#    

    def temp(alt):
        if alt < 36089.2:
            T0 = 518-3.57*alt/1000
        else:
            T0 = 386.223
        return T0

    def sigma(alt):
        if alt < 36089.2:
            p0 = 0.002377*(temp(alt)/518)**4.256
        else:
            p0 = 0.002377*(temp(alt)/518)**4.256
        return p0

    def delta(alt):
        if alt < 36089.2:
            P0 = 2116*((temp(alt)/518)**5.256)
        else:
            P0 = 2116*((temp(alt)/518)**5.256)*np.exp(-(32.2/(1716*temp(36089)))*(alt-36089))
        return P0

#----------# Freestream #--------------------------------------------------------------------------------------------------#

# T_t0, P_t0
    T_t0 = temp(alt)*(1+(((dfObj.Gamma[0])-1)/2)*(M0**2)) # R
    P_t0 = delta(alt)*(1+((dfObj.Gamma[0]-1)/2)*(M0**2))**((dfObj.Gamma[0]/(dfObj.Gamma[0]-1))) # psf
    
#----------# Inlet/Diffuser #----------------------------------------------------------------------------------------------#

# T_t2, P_t2
    T_t2 = T_t0 # R
    P_t2 = P_t0 # psf

#----------# Fan #---------------------------------------------------------------------------------------------------------#

# T_t13, P_t13
    T_t13 = T_t2*(FPR)**((dfObj.Gamma[1]-1)/(dfObj.Gamma[1]*dfObj.Eta_Poly[1])) # R
    P_t13 = P_t2*FPR # psf
    
# Cross-Sectional Area, A_c
    A_c = np.pi*(((D_fan/12)**2)/4)
    M2 = 0.7

# Mass Flow Rate of Air                                                   
    M_air = ((A_c*delta(alt))/(np.sqrt(temp(alt)))*(np.sqrt(gamma_air/R_ssl_air))*M2*
             (1+((gamma_air-1)/2)*M2**2)**(-(gamma_air+1)/(2*(gamma_air-1))))
                                                                                                    
# Work out of Fan                                               
    W_fan = M_air*dfObj.C_p[1]*(T_t13-T_t2) # ft-lbs/s

#----------# Compressor #--------------------------------------------------------------------------------------------------#
    
# P_t3, T_t3                                                   
    P_t3 = delta(alt)*OPR # psf
    T_t3 = T_t13*(P_t3/P_t13)**((dfObj.Gamma[2]-1)/(dfObj.Gamma[2]*dfObj.Eta_Poly[2])) # R

# Mass Flow Through Core                                                   
    M_core = M_air*(1/(BPR+1))

# Work Into Compressor                                                   
    W_comp = M_core*dfObj.C_p[2]*(T_t3-T_t13) # ft-lbs/s

#----------# Combustor #---------------------------------------------------------------------------------------------------#
    
# 4% Total Pressure Loss, P_t4/P_t3 = 0.96
    Pt4_Pt3 = 0.96
    P_t4 = P_t3*Pt4_Pt3 # psf

 #----------# Turbine #-----------------------------------------------------------------------------------------------------#   
    
# Fuel-Air Ratio (f)
    hPR = 14.3*10**6 # ft-lbs/lbm
    f_ratio = (dfObj.C_p[3]*T_t4-C_P_air*T_t3)/((hPR*32.2)-dfObj.C_p[3]*T_t4)
    
# Turbine Work
    W_turb = W_fan+W_comp # ft-lbs/s

# T_t5, P_t5
    T_t5 = T_t4-(W_turb/(M_core*(1+f_ratio)*dfObj.C_p[4])) # R
    P_t5 = P_t4*(T_t5/T_t4)**(dfObj.Gamma[4]/(dfObj.Eta_Poly[4]*(dfObj.Gamma[4]-1))) # psf

#----------# Fan Nozzle #--------------------------------------------------------------------------------------------------#    
    
# T_t19, P_19, T_19
    T_t19 = T_t13 # R
    P_19 = delta(alt) # psf
    T_19 = T_t13*(P_19/P_t13)**((gamma_air-1)/gamma_air) # R
                                                   
# U_19
    U_19 = np.sqrt((T_t19-T_19)/(1/(2*C_P_air))) # ft/s

#----------# Core Nozzle #-------------------------------------------------------------------------------------------------#    
    
# T_t9, P_9, T_9
    T_t9 = T_t5 # R
    P_9 = delta(alt) # psf
    T_9 = T_t9*(P_9/P_t5)**((dfObj.Gamma[5]-1)/dfObj.Gamma[5]) # R
                                                   
# U_9
    U_9 = np.sqrt(2*dfObj.C_p[5]*(T_t9-T_9)) # ft/s

#----------# Thrust #------------------------------------------------------------------------------------------------------#  

# Mass Flow Rate of Bypass
    M_bypass = BPR*M_core
    
# U0 from Mach Values
    U0 =  M0*((gamma_air*R_ssl_air*(temp(alt)))**(1/2)) # ft/s
    
# Calculate Thrust, Mass of Fuel, TSFC for each input deck
    Thr = M_core*(1+f_ratio)*U_9 + M_bypass*U_19 - (M_core+M_bypass)*U0
    M_fuel = f_ratio*M_air
    TSFC = (M_fuel/Thr)*32.2*360
    
#----------# Fancy Outputs #-----------------------------------------------------------------------------------------------#

#     print('T_t0 is ' + str(T_t0) + ' R')
#     print('P_t0 is ' + str(P_t0) + ' psf')    
#     print('M_air is ' + str(M_air) + ' sl/s')
#     print('T_t2 is ' + str(T_t2) + ' R')
#     print('P_t2 is ' + str(P_t2) + ' psf')
#     print('T_t13 is ' + str(T_t13) + ' R')
#     print('P_t13 is ' + str(P_t13) + ' psf')
#     print('W_fan is ' + str(W_fan) + ' ft-lbs/s')
#     print('T_t3 is ' + str(T_t3) + ' R')
#     print('P_t3 is ' + str(P_t3) + ' psf')
#     print('W_comp is ' + str(W_comp) + ' ft-lbs/s')
#     print('P_t4 is ' + str(P_t4) + ' psf')
#     print('fuel-ratio is ' + str(f_ratio))
#     print('T_t5 is ' + str(T_t5) + ' R')
#     print('P_t5 is ' + str(P_t5) + ' psf')
#     print('T_t19 is ' + str(T_t19) + ' R')
#     print('T_19 is ' + str(T_19) + ' R')
#     print('U_19 is ' + str(U_19) + ' ft/s')
#     print('T_9 is ' + str(T_9) + ' R')
#     print('U_9 is ' + str(U_9) + ' ft/s')
#     print('Static Thrust is ' + str(Thr_static) + ' lbs')
#     print('TSFC is ' + str(TSFC) + ' lbm/lbs-hr')
#     print(U0_static)

    return P_t0, T_t0, P_t2, T_t2, T_t13, P_t13, W_fan, P_t3, T_t3, W_comp, P_t4, f_ratio, P_t5, T_t5, U_19, T_19, U_9, T_9, Thr, TSFC

#----------# Engine Inputs #-----------------------------------------------------------------------------------------------#

# Comment out a particular Engine_Design "for" loop to see the results for each with "Fancy Outputs"

# Below includes the conditions for each input deck interated by...

'''
Maximum Conditions: Static, Sea Level (Takeoff)
Altitude: 0 - 44,000_ft
Mach #: 0
BPR: 3.7
FPR: 1.58
OPR: 40
T_t4: 3400_R
D_fan: 51.1811_ft
'''
Engine_Output_1 = []
Thr_1 = []
TSFC_1 = []
for i in np.linspace(0,44000,100):
    inputs_1 = EngineDesign(i,0,3.7,1.58,40,3400,51.1811)
    Engine_Output_1.append(inputs_1)
    Thr_1.append(inputs_1[18])
    TSFC_1.append(inputs_1[19])

'''
Design Conditions: Cruise @ 44,000_ft
Altitude: 0 - 44,000_ft
Mach #: 0.8
BPR: 3.7
FPR: 1.58
OPR: 40
T_t4: 3150_R
D_fan: 51.1811_ft
'''
Engine_Output_2 = []
Thr_2 = []
TSFC_2 = []
for i in np.linspace(0,44000,100):
    inputs_2 = EngineDesign(i,0.8,3.7,1.58,40,3150,51.1811)
    Engine_Output_2.append(inputs_2)
    Thr_2.append(inputs_2[18])
    TSFC_2.append(inputs_2[19])

'''
Maximum Conditions: Static, Sea Level (Takeoff)
Altitude: 44,000_ft
Mach #: 0 - 0.9
BPR: 3.7
FPR: 1.58
OPR: 40
T_t4: 3400_R
D_fan: 51.1811_ft
'''
Engine_Output_3 = []
Thr_3 = []
TSFC_3 = []
for i in np.linspace(0,0.9,100):
    inputs_3 = EngineDesign(44000,i,3.7,1.58,40,3400,51.1811)
    Engine_Output_3.append(inputs_3)
    Thr_3.append(inputs_3[18])
    TSFC_3.append(inputs_3[19])

'''
Design Conditions: Cruise @ 44,000_ft
Altitude: 44,000_ft
Mach #: 0 - 0.9
BPR: 3.7
FPR: 1.58
OPR: 40
T_t4: 3150_R
D_fan: 51.1811_ft
'''
Engine_Output_4 = []
Thr_4 = []
TSFC_4 = []
for i in np.linspace(0,0.9,100):
    inputs_4 = EngineDesign(44000,i,3.7,1.58,40,3150,51.1811)
    Engine_Output_4.append(inputs_4)
    Thr_4.append(inputs_4[18])
    TSFC_4.append(inputs_4[19])

In [5]:
plot = True

if plot == True:
    
    # Thurst v. Altitude Plot
    plt.plot(np.linspace(0,44000,100), Thr_static_1, 'cadetblue', label = "Maximum")
    plt.plot(np.linspace(0,44000,100), Thr_static_2, 'blueviolet', label = "Design")
    plt.title('Thrust v. Altitude', fontsize = 15)
    plt.xlabel('Altitude (ft)', fontsize=12)
    axes = plt.gca()
    axes.set_xlim([0,44000])
    plt.ylabel('Thrust', fontsize=12)
    plt.tight_layout()
    plt.legend(loc="upper right")
    #plt.savefig('AE_425_Engine_Project_Thrust_1.png')
    plt.show()
    
    # TSFC v. Altitude Plot
    plt.plot(np.linspace(0,44000,100), TSFC_1, 'cadetblue', label = "Maximum")
    plt.plot(np.linspace(0,44000,100), TSFC_2, 'blueviolet', label = "Design")
    plt.title('TSFC v. Altitude', fontsize = 15)
    plt.xlabel('Altitude (ft)', fontsize=12)
    axes = plt.gca()
    axes.set_xlim([0,44000])
    plt.ylabel('TSFC', fontsize=12)
    plt.tight_layout()
    plt.legend(loc="center left")
    #plt.savefig('AE_425_Engine_Project_TSFC_1.png')
    plt.show()
    
    # Thurst v. Mach Plot
    plt.plot(np.linspace(0,0.9,100), Thr_static_3, 'cadetblue', label = "Maximum")
    plt.plot(np.linspace(0,0.9,100), Thr_static_4, 'blueviolet', label = "Design")
    plt.title('Thrust v. Mach', fontsize = 15)
    plt.xlabel('Mach', fontsize=12)
    axes = plt.gca()
    #axes.set_xlim([0,0.15])
    #plt.yscale('log')
    plt.ylabel('Thrust', fontsize=12)
    plt.tight_layout()
    plt.legend(loc="upper right")
    #plt.savefig('AE_425_Engine_Project_Thrust_2.png')
    plt.show()
    
    # TSFC v. Mach Plot
    plt.plot(np.linspace(0,0.9,100), TSFC_3, 'cadetblue', label = "Maximum")
    plt.plot(np.linspace(0,0.9,100), TSFC_4, 'blueviolet', label = "Design")
    plt.title('TSFC v. Mach', fontsize = 15)
    plt.xlabel('Mach', fontsize=12)
    axes = plt.gca()
    #axes.set_ylim([0.338,.34])
    #plt.yscale('log')
    plt.ylabel('TSFC', fontsize=12)
    plt.tight_layout()
    plt.legend(loc="upper left")
    #plt.savefig('AE_425_Engine_Project_TSFC_2.png')
    plt.show()

NameError: name 'Thr_static_1' is not defined

In [3]:
dfObj

,Eta_Poly,Gamma,R,C_p
Inlet,1.00,1.40,1716,6006.000000
Fan,0.97,1.40,1716,6006.000000
Compressor,0.90,1.40,1716,6006.000000
Combustor,0.00,1.33,1775,7153.787879
Turbine,0.91,1.33,1775,7153.787879
Nozzle (Core),1.00,1.33,1775,7153.787879
Nozzle (Bypass),1.00,1.40,1716,6006.000000
